<a href="https://colab.research.google.com/github/neerajguleria1/driver-drowsiness-detection/blob/main/Untitled13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
%%writefile temporal_Risk.py
import numpy as np
from collections import deque

class TemporalRiskTracker:
  def __init__(self,window=5):
    self.window=window
    self.risk_history=deque(maxlen=window)


  def update(self,risk_score:float)->dict:

    self.risk_history.append(risk_score)
    history=np.array(self.risk_history)

    smoothed_risk=history.mean()
    trend=self._compute_trend(history)

    return {
        "current_risk":risk_score,
        "smoothed_risk":smoothed_risk,
        "trend":trend
    }

  def _compute_trend(self,history:np.ndarray) -> str:
    if len(history)<3:
      return "INSUFFICIENT_DATA"

    slope=np.polyfit(range(len(history)),history,1)[0]

    if slope>2:
      return "RISING"
    elif slope<-2:
      return "FALLING"
    else:
      return "STABLE"


Writing temporal_Risk.py


In [36]:
%%writefile cognitive_engine.py

"""
Cognitive Risk Engine

Purpose:
- Convert ML drowsiness probability into a explainable rish score
- Focus ML output with physocological + behavioural signals.
- Provide stable, decision-ready rish states
"""

import numpy as np
import pandas as pd

class CognitiveRiskEngine:
  REQUIRED_COLUMNS=[
      "Fatigue",
      "Alertness",
      "prev_alertness",
      "HR",
      "Speed",
      "speed_change"
  ]
  def __init__(self):
    self.prev_risk=0.0

  def compute(self,df:pd.DataFrame,ml_prob:float)->dict:
    missing=set(self.REQUIRED_COLUMNS) - set(df.columns)
    if missing:
      raise ValueError(f"Missing required input columns: {missing}")

    fatigue=df["Fatigue"].iloc[0] # iloc[0] give me the the actual number from the first row, not the whole column
    alertness=df["Alertness"].iloc[0]
    prev_alertness=df["prev_alertness"].iloc[0]
    hr=df["HR"].iloc[0]
    speed=df["Speed"].iloc[0]
    speed_change=df["speed_change"].iloc[0]

    fatigue_score=np.clip(fatigue/10,0,1)
    alertness_low=np.clip(1-alertness,0,1)

    alertness_drop=1.0 if (prev_alertness-alertness) > 0.2 else 0.0

    hr_stress=np.clip((hr-90)/30,0,1)
    speed_instability=np.clip(speed_change/30,0,1)


    raw_risk = (
        0.35*ml_prob
        +0.20*fatigue_score
        +0.15*alertness_low
        +0.10*alertness_drop
        +0.10*hr_stress
        +0.10*speed_instability
    )

    smoothed_risk=0.7*self.prev_risk +0.3*raw_risk
    self.prev_risk=smoothed_risk

    risk_score=int(np.clip(smoothed_risk*100,0,100))

    if risk_score<30:
      state="SAFE"
      action="No alert"
    elif risk_score<60:
      state="CAUTION"
      action="Audio warning"
    else:
      state="CRITICAL"
      action="Audio + Visual + Haptic alert"


    return {
        "risk_score":risk_score,
        "state":state,
        "recommended_action":action,
        "explanation":{
            "ml_probability":round(ml_prob,2),
            "fatigue_score":round(fatigue_score,2),
            "alertness_low":round(alertness_low,2),
            "hr_stress":round(hr_stress,2),
            "speed_instability":round(speed_instability,2)
        }
    }

Overwriting cognitive_engine.py


In [40]:
import joblib
import pandas as pd
from google.colab import files
#uploaded=files.upload()
from cognitive_engine import CognitiveRiskEngine
from temporal_Risk import TemporalRiskTracker

pipeline=joblib.load("final_driver_drowsiness_pipeline.pkl")

cognitive=CognitiveRiskEngine()
temporal=TemporalRiskTracker()

samples=[
    {"Speed":70,"Alertness":0.7,"Seatbelt":1,"HR":85,"Fatigue":4,"speed_change":13,"prev_alertness":0.75},
    {"Speed": 65, "Alertness": 0.6, "Seatbelt": 1, "HR": 95, "Fatigue": 5, "speed_change": 9, "prev_alertness": 0.7},
    {"Speed": 60, "Alertness": 0.5, "Seatbelt": 1, "HR": 102, "Fatigue": 6, "speed_change": 6, "prev_alertness": 0.6},
    {"Speed": 58, "Alertness": 0.45, "Seatbelt": 1, "HR": 108, "Fatigue": 7, "speed_change": 5, "prev_alertness": 0.55},
    {"Speed": 55, "Alertness": 0.4, "Seatbelt": 1, "HR": 112, "Fatigue": 8, "speed_change": 4, "prev_alertness": 0.5},
]

for i, samples in enumerate(samples):
  df=pd.DataFrame([samples])

  ml_prob=pipeline.predict_proba(df)[0][1]
  cognitive_result=cognitive.compute(df,ml_prob)
  temporal_result=temporal.update(cognitive_result["risk_score"])

print(f"/nStep {i+1}")
print("Cognitive:",cognitive_result)
print("Temporal:",temporal_result)

/nStep 5
Cognitive: {'risk_score': 38, 'state': 'CAUTION', 'recommended_action': 'Audio warning', 'explanation': {'ml_probability': np.float64(0.62), 'fatigue_score': np.float64(0.8), 'alertness_low': np.float64(0.6), 'hr_stress': np.float64(0.73), 'speed_instability': np.float64(0.13)}}
Temporal: {'current_risk': 38, 'smoothed_risk': np.float64(24.0), 'trend': 'RISING'}
